# CNN

In [3]:
from tensorflow.examples.tutorials.mnist import mnist, input_data

data_sets = input_data.read_data_sets("/tmp/data/", one_hot=False)

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

Extracting /tmp/data/train-images-idx3-ubyte.gz


Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
learning_rate = 0.001
num_steps = 2000
batch_size = 128

num_input = mnist.IMAGE_PIXELS
num_classes = mnist.NUM_CLASSES
dropout = 0.25

In [7]:
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    with tf.variable_scope('ConvNet', reuse=reuse):
        x = x_dict['images']
        # [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, mnist.IMAGE_SIZE, mnist.IMAGE_SIZE, 1])
        
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
        
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        
        fc1 = tf.contrib.layers.flatten(conv2)
        fc1 = tf.layers.dense(fc1, 1024)
        
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
        
        out = tf.layers.dense(fc1, n_classes)
        
        return out

In [14]:
def model_fn(features, labels, mode):
    logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)
    
    pred_classes = tf.argmax(logits_test, axis=1)
    # pred_probas = tf.nn.softmax(logits_test)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits = logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op}
    )
    return estim_specs

In [15]:
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.


In [16]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': data_sets.train.images},
    y=data_sets.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True
)

model.train(input_fn, steps=num_steps)

INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp3atjgn_v/model.ckpt.


INFO:tensorflow:loss = 2.32253, step = 1


INFO:tensorflow:global_step/sec: 10.3689


INFO:tensorflow:loss = 0.17832, step = 101 (9.645 sec)


...

INFO:tensorflow:loss = 0.0282995, step = 1801 (13.173 sec)


INFO:tensorflow:global_step/sec: 9.86602


INFO:tensorflow:loss = 0.0309088, step = 1901 (10.136 sec)


INFO:tensorflow:Saving checkpoints for 2000 into /tmp/tmp3atjgn_v/model.ckpt.


INFO:tensorflow:Loss for final step: 0.0532318.


In [18]:
# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': data_sets.test.images}, y=data_sets.test.labels,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn)

INFO:tensorflow:Starting evaluation at 2018-01-07-17:44:55


INFO:tensorflow:Restoring parameters from /tmp/tmp3atjgn_v/model.ckpt-2000


INFO:tensorflow:Finished evaluation at 2018-01-07-17:44:58


INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9899, global_step = 2000, loss = 0.032857


{'accuracy': 0.98989999, 'global_step': 2000, 'loss': 0.032856956}

In [ ]:
# Predict single images
%matplotlib inline
n_images = 4
# Get images from test set
test_images = data_sets.test.images[:n_images]
# Prepare the input data
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': test_images}, shuffle=False)
# Use the model to predict the images class
preds = list(model.predict(input_fn))

# Display
for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
    plt.show()
    print("Model prediction:", preds[i])

INFO:tensorflow:Restoring parameters from /tmp/tmp3atjgn_v/model.ckpt-2000


Model prediction: 7
Model prediction: 2


Model prediction: 1
Model prediction: 0
